In [1]:
import openai
import pandas as pd
import seaborn as sns
import time
import os

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
## access the posts csvs
post_list_1 = pd.read_csv(r'data\refused_post_list.csv').Text.tolist()
post_list_2 = pd.read_csv(r'data\approved_post_list.csv').Text.tolist()
post_list_3 = pd.read_csv(r'data\refused_post_list.csv').Text.tolist()
post_list_4 = pd.read_csv(r'data\rejected_post_list.csv').Text.tolist()
post_list = post_list_1 + post_list_2 + post_list_3 + post_list_4

In [3]:
post_list = pd.read_csv(r'data\all_posts_1.csv')["0"].tolist()

In [4]:
### remove posts that are too short or too long
for post in post_list:
        if len(post) < 250 or len(post) > 6000:
            post_list.remove(post)

In [17]:
messages = []
responses = []
disclaimer = "One word answers only. Answer in the following format. Program: Degree Level: Univsersity: Sponsor: Scholarship: Relatives: Visa Attempt: Visa Result:"  
system_message = "You are a keyword extractor! Your job is to answer in the following question, strictly speaking, in only one word: What program is applicant applying for? What degree level is he applying or, bachelors, masters or phd? Which university is he applying visa to? Who is sponsoring the applicant? Did he recieve any scholarship? Did he had any relatives in US? How many visa attempts he has had(if not mentioned, it's first)? Was his visa approved or refused? If not applicable, say 'N/A' Program: Degree Level: University: Sponsor: Scholarship: Relatives: Visa Attempts: Visa Status:(Consider 'Refused under 221g' as 'Approved')"
sample_question = "Visa interview Islamabad Texas A&M university 5th Aug: 11:30am Vo: Hello Me:hello sir  Vo: i need u to do your finger prints. (Messed up with left hand once) Me:done Vo: alright pass me those documents. Me: here Vo: so why are you going to the Us? Me: I'm going to pursue my bachelor's degree in health sciences at texas a&m university. Vo: how many universities did you apply to? Me: about 13 universities, I got accepted from a few. Vo: and which ones did u get accepted from? Me: i got accepted from the university of Hartford, texas a&m and university of---uh and I had a correspondence with the Baylor university. Vo: ok, so why did you choose this university? Me: well it has quite good ranking regarding my course, its listed as the 28th best medical university in terms of health care management and in biological scien--- Vo: alright so do you have any relatives in the Us? Me: Yes, an uncle of mine lives there with his family. Vo: where do they live? Me: Irving (accident said the old address) Vo: California?-- what state? Me: no not California, its in Dallas at Texas. Vo: alright and what does he do? Me: his work is associated with transport but I don't really know much about that. Vo: oh okay, so whose going to sponsor you? Me: my father will Vo: and what does he do? Me: told Vo: (going though my docs) have you ever been to the Us before? Me: yes, twice Vo: okay. (Keeps typing and looking through stuff for about a minute or two). Vo: alright, I'm approving your visa, but we'll be keeping your passport for further processing. You will receive it within 1-2weeks. There were some 30 candidates I saw that day and out of them only 2 girls & me myself got approved. One common thing between me and one of those girls was that we both had been to the Us before. Me twice for visiting and her for her bachelors(now going for masters I believe)."
sample_answer = "Program: Health Sciences Degree Level: Bachelors University: Texas A&M University Sponsor: Father Scholarship: N/A Relatives: Uncle Visa Attempts: First Visa Result: Approved"

for i in range(31+22+92+15, len(post_list)):
    time.sleep(1) # to prevent API from crashing
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": system_message},
                {"role": "user", "content": sample_question},
                {"role": "assistant", "content": sample_answer},
               {"role": "user", "content": disclaimer + post_list[i]}],
        temperature = 0
    )
    responses.append(completion.choices[0].message.content)

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [16]:
print(f"The number of data we have is : {len(responses)}")
final_data = [] # Comment out if you want to append to existing data

The number of data we have is : 5


In [10]:
str(responses[0].split(": ")[3]).replace("Sponsor", "").strip()
columns = ["Program", "Degree Level", "University", "Sponsor", "Scholarship", "Relatives", "Visa Attempt", "Visa Result", "Id"]

for i in range(len(responses)):
    values = str(responses[i]).split(": ")
    if len(values) != len(columns):
        print(f"Skipping Response Number: {i+1} *** Response: {responses[i][0:30]}...")
        continue
    data_dict = {}
    for j in range(0, len(columns)):
        key = columns[j-1]
        value = values[j].replace(columns[j], "").strip()
        data_dict[key] = value

    final_data.append(data_dict)
    

In [11]:
final_data
df = pd.DataFrame(final_data)
print(df.shape)
df.head()

(15, 9)


,Id,Program,Degree Level,University,Sponsor,Scholarship,Relatives,Visa Attempt,Visa Result
0,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,,N/A,Masters,University of Cumberlands,Brother,N/A,Sisters and Brother\ns,First,Approved
2,,Chemistry,PhD,University of Tulsa,N/A,Research Assistantship,N/A,N/A,N/A
3,,Public Health,Masters,N/A,Parents,N/A,Brother,First,Approved
4,,LLM,Masters,N/A,N/A,Fully/Partially Funded,N/A,N/A,N/A


In [12]:
### Saving the data
df.to_csv(r"data\training_data_new_2.csv")